In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
import pickle
import matplotlib.pyplot as plt
from tool import *

Using TensorFlow backend.


In [2]:
mnist = keras.datasets.mnist
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()
def MNIST_To_CIFAR_FORM(mnist_train_x, mnist_train_y,mnist_test_x, mnist_test_y):
    """
    Change the one-channel to RBG-channel on mnist_train_x and mnist_test_x
    Change the shape of mnist_train_y and mnist_test_y from (length) to (length,1)
    ---------------------------------------
    inputs:
    mnist_train_x, mnist_train_y,mnist_test_x, mnist_test_y which is all multi-dimension array
    It is recommended to use the following way to import the data
    ========================== codes ==========================
    mnist = keras.datasets.mnist
    (mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()
    ========================== codes ==========================
    outputs:
    mnist_train_RGB_x, M_train_y, mnist_test_RGB_x, M_test_y 
    """
    from skimage import exposure
    import imutils
    B= []
    for i in range(len(mnist_train_x)):
        A = mnist_train_x[i]
        A = exposure.rescale_intensity(A, out_range=(0, 255))
        A = imutils.resize(A, width=32)
        B.append(A)
    B = np.array(B)

    mnist_train_RGB_x = np.repeat(B[:,:, :, np.newaxis], 3, axis=3)
    B= []
    for i in range(len(mnist_test_x)):
        A = mnist_test_x[i]
        A = exposure.rescale_intensity(A, out_range=(0, 255))
        A = imutils.resize(A, width=32)
        B.append(A)
    B = np.array(B)

    mnist_test_RGB_x = np.repeat(B[:,:, :, np.newaxis], 3, axis=3)
    M_train_y = np.array([[mnist_train_y[i]] for i in range(len(mnist_train_y))])
    M_test_y = np.array([[mnist_test_y[i]] for i in range(len(mnist_test_y))])
    return mnist_train_RGB_x, M_train_y, mnist_test_RGB_x, M_test_y

mnist_train_RGB_x, M_train_y, mnist_test_RGB_x, M_test_y = MNIST_To_CIFAR_FORM(mnist_train_x, mnist_train_y,mnist_test_x, mnist_test_y)
(C_x_train, C_y_train), (C_x_test, C_y_test) = cifar10.load_data()

(fm_x_train, fm_y_train), (fm_x_test, fm_y_test) = keras.datasets.fashion_mnist.load_data()
fm_train_RGB_x, fm_train_y, fm_test_RGB_x, fm_test_y = MNIST_To_CIFAR_FORM(fm_x_train, fm_y_train,fm_x_test, fm_y_test)

In [3]:
class MNIST_vgg:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('MNIST_vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 25
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        x_train,y_train,x_test,y_test = mnist_train_RGB_x,M_train_y,mnist_test_RGB_x,M_test_y
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        model.save_weights('MNIST_vgg.h5')
        return model
    
class fmvgg:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('fmvgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = np.mean(x)
        std = np.std(x)
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 300
        learning_rate = 0.001
        lr_decay = 1e-5
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        x_train, y_train, x_test, y_test = fm_train_RGB_x, fm_train_y, fm_test_RGB_x, fm_test_y
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        # sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        sgd = optimizers.adam(lr=learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        model.save_weights('fmvgg.h5')
        return model

In [4]:
fp = open("MNIST-VGG-3.pkl","rb+")
M_VGG_Model3 = pickle.load(fp, encoding='bytes')
fp = open("FM-VGG-3.pkl","rb+")
FM_VGG_Model3 = pickle.load(fp, encoding='bytes')
fp = open("FM_representation_result_IDFM.pkl","rb+")
FM_representation_result_IDFM = pickle.load(fp, encoding='bytes')
fp = open("FM_representation_result_ODM.pkl","rb+")
FM_representation_result_ODM = pickle.load(fp, encoding='bytes')
fp = open("M_representation_result_ODFM.pkl","rb+")
M_representation_result_ODFM = pickle.load(fp, encoding='bytes')
fp = open("M_representation_result_IDM.pkl","rb+")
M_representation_result_IDM = pickle.load(fp, encoding='bytes')

W1112 02:17:47.689144 27164 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1112 02:17:47.702110 27164 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1112 02:17:47.723054 27164 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1112 02:17:47.724053 27164 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is depre

In [5]:
def concact_results_OD(one_sample,arr_KNN_from_same_class_ratio, arr_KNN_max_class_label,MODEL):
    pred_labels = MODEL.predict(one_sample[:500])
    import pandas as pd
    df = pd.DataFrame({'arr_KNN_max_class_label':arr_KNN_max_class_label,
                   'arr_KNN_from_same_class_ratio':arr_KNN_from_same_class_ratio,
                   'predicted_label':find_statistics(pred_labels)[2],
                  'predicted_prob':find_statistics(pred_labels)[1],})
    return df
def concact_results_ID(one_sample,ground_truth_one_sample, arr_KNN_from_same_class_ratio, arr_KNN_max_class_label,MODEL):
    pred_labels = MODEL.predict(one_sample[:500])
    ground_truth_one_sample = ground_truth_one_sample.reshape(-1)
    import pandas as pd
    df = pd.DataFrame({'arr_KNN_max_class_label':arr_KNN_max_class_label,
                   'arr_KNN_from_same_class_ratio':arr_KNN_from_same_class_ratio,
                   'predicted_label':find_statistics(pred_labels)[2],
                  'predicted_prob':find_statistics(pred_labels)[1],
                'ground_truth':ground_truth_one_sample[:500]})
    return df

In [6]:
def consistency_of_df(df, upper_bound, lower_bound):
    return len(df[df['arr_KNN_from_same_class_ratio']>upper_bound][df['arr_KNN_max_class_label'] == df['predicted_label']])/len(df[df['arr_KNN_from_same_class_ratio']>upper_bound]), len(df[df['arr_KNN_from_same_class_ratio']<lower_bound][df['arr_KNN_max_class_label'] == df['predicted_label']])/len(df[df['arr_KNN_from_same_class_ratio']<lower_bound])

In [7]:
df_M_representation_result_ODFM = concact_results_OD(fm_train_RGB_x,M_representation_result_ODFM[1],M_representation_result_ODFM[2],M_VGG_Model3)

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node activation_1/Relu}}]]
	 [[activation_15/Softmax/_3155]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node activation_1/Relu}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
df_M_representation_result_ODFM[df_M_representation_result_ODFM['arr_KNN_from_same_class_ratio']>0.7]

In [11]:
len(df_M_representation_result_ODFM[df_M_representation_result_ODFM['arr_KNN_from_same_class_ratio']>0.7])

231

In [12]:
len(df_M_representation_result_ODFM[df_M_representation_result_ODFM['arr_KNN_from_same_class_ratio']>0.7][df_M_representation_result_ODFM['arr_KNN_max_class_label'] == df_M_representation_result_ODFM['predicted_label']])

c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


41

In [13]:
df_M_representation_result_ODFM[df_M_representation_result_ODFM['arr_KNN_from_same_class_ratio']<0.4]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob
5,4,0.32,0,0.931305
9,2,0.24,5,0.223131
13,0,0.38,0,0.869029
26,8,0.22,1,0.405436
30,1,0.32,2,0.828062
43,8,0.32,0,0.716087
47,0,0.38,7,0.754881
57,0,0.38,6,0.437267
70,2,0.22,2,0.493238
92,1,0.24,7,0.955604


In [14]:
df_M_representation_result_IDM = concact_results_ID(mnist_test_RGB_x,M_test_y,M_representation_result_IDM[1],M_representation_result_IDM[2],M_VGG_Model3)

In [15]:
df_M_representation_result_IDM[df_M_representation_result_IDM['arr_KNN_from_same_class_ratio']>0.7]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob,ground_truth
0,7,1.00,7,0.999994,7
1,2,1.00,2,0.999490,2
2,1,1.00,1,0.999907,1
3,0,1.00,0,0.999972,0
4,4,1.00,4,0.999983,4
5,1,1.00,1,0.999893,1
6,4,0.98,4,0.999968,4
7,9,0.98,9,0.996378,9
9,9,1.00,9,0.999572,9
10,0,1.00,0,0.999984,0


In [16]:
df_M_representation_result_IDM[df_M_representation_result_IDM['arr_KNN_from_same_class_ratio']<0.4]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob,ground_truth
62,9,0.34,9,0.530925,9
78,0,0.34,9,0.669924,9
95,8,0.34,4,0.999936,4
97,7,0.38,7,0.999799,7
243,7,0.32,7,0.999830,7
460,2,0.34,5,0.983897,5
495,8,0.38,8,0.997395,8


In [19]:
df_FM_representation_result_ODM = concact_results_OD(mnist_train_RGB_x,FM_representation_result_ODM[1],FM_representation_result_ODM[2],FM_VGG_Model3)

In [20]:
df_FM_representation_result_ODM[df_FM_representation_result_ODM['arr_KNN_from_same_class_ratio']>0.7]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob
5,9,0.72,8,0.974921
8,1,0.96,1,0.926724
21,8,0.84,8,0.920886
23,3,0.72,1,0.952120
40,1,1.00,1,0.998118
56,8,0.76,0,0.443508
82,9,0.86,2,0.385207
102,1,0.86,1,0.984364
104,1,0.86,1,0.987287
112,1,0.96,1,0.991701


In [21]:
df_FM_representation_result_ODM[df_FM_representation_result_ODM['arr_KNN_from_same_class_ratio']<0.4]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob
4,1,0.22,8,0.744535
6,1,0.30,1,0.996362
7,2,0.32,8,0.883240
13,3,0.38,0,0.991715
15,6,0.22,8,0.935659
16,2,0.18,8,0.868039
17,4,0.36,8,0.884808
27,8,0.36,8,0.927261
31,4,0.34,8,0.848993
32,3,0.28,0,0.985358


In [22]:
df_FM_representation_result_IDFM = concact_results_ID(fm_test_RGB_x,fm_test_y,FM_representation_result_IDFM[1],FM_representation_result_IDFM[2],FM_VGG_Model3)

In [23]:
df_FM_representation_result_IDFM[df_FM_representation_result_IDFM['arr_KNN_from_same_class_ratio']>0.7]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob,ground_truth
2,1,1.00,1,0.999982,1
3,1,0.98,1,0.999995,1
5,1,1.00,1,0.999993,1
6,1,0.98,4,0.997872,4
8,5,0.84,5,0.999892,5
9,7,0.92,7,0.999997,7
11,5,0.76,5,0.999852,5
13,3,0.74,3,0.997555,3
15,1,0.96,1,0.999995,1
19,0,0.76,0,0.999496,0


In [24]:
df_FM_representation_result_IDFM[df_FM_representation_result_IDFM['arr_KNN_from_same_class_ratio']<0.4]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob,ground_truth
18,6,0.32,8,0.999910,8
23,0,0.22,9,0.594272,9
29,4,0.32,3,0.997260,3
33,4,0.38,3,0.997714,3
34,0,0.30,8,0.999987,8
40,4,0.38,6,0.999169,6
48,5,0.28,2,0.929888,2
51,5,0.26,4,0.964323,4
66,2,0.26,2,0.789516,2
67,3,0.38,3,0.962343,3


In [25]:
df_s = [df_M_representation_result_ODFM, df_M_representation_result_IDM, df_FM_representation_result_ODM, df_FM_representation_result_IDFM]

In [26]:
for df in df_s:
    print(consistency_of_df(df))

(0.1774891774891775, 0.1388888888888889)
(0.9705882352941176, 0.5714285714285714)
(0.3235294117647059, 0.16393442622950818)
(0.9375, 0.3595505617977528)


c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
